# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data[['race','call']].head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [7]:
w = data[data.race=='w'][['race','call']]
b = data[data.race=='b'][['race','call']]

print(data.shape,w.shape,b.shape)
print(w.call.sum(),b.call.sum())

(4870, 65) (2435, 2) (2435, 2)
235.0 157.0


### 1. What test is appropriate for this problem? Does CLT apply?

This problem requires a hypothesis test for the difference in proportions between two samples. The samples are large enough for the CLT to apply.

### 2. What are the null and alternate hypotheses?

We'll use the null hypothesis that there is no difference in proportion between the two populations (i.e. there is no racial discrimination here). And the alternative hypothesis is that the white-sounding names have a higher callback rate than black sounding names.

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [28]:
#assign sample sizes and proportions to variables
n1=w.shape[0]
n2=b.shape[0]
p1=w.call.sum()/n1
p2=b.call.sum()/n2
prop_diff=p1-p2

#calculate the margin of error of the pooled proportion from the null proportion 0.5
pool_p=(p1*n1+p2*n2)/(n1+n2)
margin_of_error=np.sqrt(pool_p*(1-pool_p)*(n1+n2)/(n1*n2))

print('Sample Proportion Difference: ',prop_diff)
print('Margin of Error: ', margin_of_error)

#Bootstrap approach using permutation sampling

#Set the number of permutations
size=10**4

#preallocate memory for permutation samples' difference in proportion
perm_prop_diffs=np.empty(size)

#iterate 10,000 times
for i in range(size):
    #randomly permute the call column
    perm_sample=np.random.permutation(data.call)
    
    #partition into two bootstrap sets, one for w and one for b, with same size as the originals
    w_perm=perm_sample[:n1]
    b_perm=perm_sample[n1:]
    #calculate proportion difference and save in perm_prop_diffs
    perm_prop_diffs[i]=w_perm.sum()/n1-b_perm.sum()/n2

#calculate p-value: number of bootstrap samples with proportion difference greater than the original
bs_p_val=(perm_prop_diffs >= (prop_diff) ).sum()/size

#calculate 95% confidence interval
bs_conf_int=list(np.round(np.percentile(perm_prop_diffs,[2.5,97.5]),3))

print('********')
print('Bootstrap P value: ',bs_p_val)
print('Bootstrap 95% confidence interval: ',bs_conf_int)

#Frequentist approach using a z test on proportion difference

#save the value for standard deviation of a two-sample proportion test to test_sd
test_sd=np.sqrt(p1*(1-p1)/n1+p2*(1-p2)/n2)

#calculate the test statistic. a bit simpler because the the mean of the null hypothesis distribution is 0
z_score=prop_diff/test_sd

#find the p value of the test statistic using stats.norm.sf
p_val=stats.norm.sf(z_score)

#calculate frequentist confidence interval using stats.norm.ppf
conf_int=np.round(list(stats.norm.ppf([0.025,.975])*test_sd),3)

print('********')
print('Frequentist P value: ',p_val)
print('Frequentist 95% confidence interval: ',conf_int)

Sample Proportion Difference:  0.032032854209445585
Margin of Error:  0.007796894036170456
********
Bootstrap P value:  0.0
Bootstrap 95% confidence interval:  [-0.015, 0.016]
********
Frequentist P value:  1.931282603761311e-05
Frequentist 95% confidence interval:  [-0.015  0.015]


### 4. Write a story describing the statistical significance in the context or the original problem.

I have tested the statistical significance of the difference in proportion of the two samples using two methods: bootstrap permutation and a classical two-sample z test.

Both methods yielded near-identical results. With P-values so close to zero, we can reject the null hypothesis that there is no statistical difference in the number of callbacks received by resumes with white-sounding names versus those with black-sounding names. This sample is very convincing that there is racial discrimination against black people, at least for the types of jobs tested in the study.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Absolutely not. It could be, but we haven't tested any other factors. To look at other factors, I would calculate the coorelation coefficient between the call column and each other column.